In [ ]:
!pip install pingouin

import pandas as pd
import pingouin as pg
import numpy as np

def calculate_icc(csv_file):
    data = pd.read_csv(csv_file)
    return pg.intraclass_corr(data=data, targets='targets', raters='raters', ratings='ratings'), data

def calculate_sems_and_mdcs(icc_value, data, suffix):
    s = np.std(data['ratings'], ddof=1)
    sem = s * np.sqrt(1 - icc_value)
    ratings_diff = data.groupby('targets')['ratings'].diff().dropna()
    sd_diff = np.std(ratings_diff, ddof=1)
    sem_diff = sd_diff / np.sqrt(2)

    mdc95 = sem * 1.96 * np.sqrt(2)
    mdc95_diff = sem_diff * 1.96 * np.sqrt(2)

    print(f"SEM{suffix} (using ICC): {sem:.4f}")
    print(f"SEM (using SD of diffs): {sem_diff:.4f}")
    print(f"MDC95 (using ICC): {mdc95:.4f}")
    print(f"MDC95 (using SD of diffs): {mdc95_diff:.4f}")

def main_icc(csv_file):
    icc_results, data = calculate_icc(csv_file)

    for icc_type in ['ICC1', 'ICC2', 'ICC3']:
        icc_row = icc_results[icc_results['Type'] == icc_type]
        if not icc_row.empty:
            icc_value = icc_row['ICC'].values[0]
            print(f"\n{icc_type} Value: {icc_value:.4f}")
            calculate_sems_and_mdcs(icc_value, data, icc_type[-1])

# 使用例
main_icc('data_2_2.csv')
